In [1]:
import sys, os

In [3]:
import numpy as np

In [2]:
sys.path.append(os.pardir)

## 시그모이드(sigmoid)

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

## 소프트맥스(softmax)

In [8]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

## 크로스 엔트로피(cross entropy)

In [9]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

## 수치 미분

In [31]:
def _numerical_gradient_1d(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        
    return grad

In [32]:
def numerical_gradient(f, X):
    if X.ndim == 1:
        return _numerical_gradient_1d(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_1d(f, x)
        
        return grad

## 2층 신경망 설계

In [34]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [35]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

In [36]:
net.params['W1'].shape

(784, 100)

In [37]:
net.params['b1'].shape

(100,)

In [38]:
net.params['W2'].shape

(100, 10)

In [39]:
net.params['b2'].shape

(10,)

## 예측 예시

In [40]:
x = np.random.rand(100, 784)
y = net.predict(x)

In [41]:
y

array([[0.00103484, 0.0010374 , 0.00097015, 0.00102007, 0.00095289,
        0.00099495, 0.00093266, 0.00099744, 0.00098791, 0.00106811],
       [0.00103703, 0.00103539, 0.00096894, 0.00102082, 0.00095795,
        0.00099674, 0.00093409, 0.00099814, 0.0009885 , 0.00106925],
       [0.00103614, 0.00103423, 0.00097308, 0.00101991, 0.00095821,
        0.00099513, 0.00093452, 0.00099596, 0.0009883 , 0.00106581],
       [0.00103457, 0.00103534, 0.00096725, 0.00102275, 0.00095844,
        0.0009967 , 0.00093319, 0.00099357, 0.0009882 , 0.0010668 ],
       [0.00103513, 0.00103311, 0.00096905, 0.00101822, 0.0009582 ,
        0.00099502, 0.00093226, 0.00099959, 0.00098967, 0.00106608],
       [0.00103924, 0.00103538, 0.0009717 , 0.00102061, 0.00095298,
        0.00099282, 0.00093352, 0.00099691, 0.00098781, 0.00107154],
       [0.00103751, 0.00103331, 0.00097025, 0.00102508, 0.00095461,
        0.00099393, 0.00093148, 0.00099573, 0.00099031, 0.00106824],
       [0.00103886, 0.00103397, 0.0009708

## 기울기(gradient) 예시

In [42]:
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

grads = net.numerical_gradient(x, t)

In [55]:
grads['W1'].shape

(784, 100)

In [56]:
grads['b1'].shape

(100,)

In [43]:
grads

{'W1': array([[-1.70458998e-04, -4.77897366e-05, -2.09994644e-04, ...,
         -2.98677123e-04, -2.25999175e-05,  5.20470023e-05],
        [-2.85778836e-04,  2.13289972e-04, -1.44624681e-04, ...,
         -1.23329400e-04,  4.77504081e-05, -2.12769269e-05],
        [-7.26898985e-05, -5.17221022e-05, -1.32748674e-04, ...,
         -2.04211559e-04, -6.63163657e-05,  4.16743973e-05],
        ...,
        [ 2.13296580e-05,  8.97586094e-05,  1.47201762e-05, ...,
         -1.10514367e-04,  4.35668301e-05,  1.67953571e-04],
        [-7.77862574e-05,  5.17856336e-05, -6.01331873e-05, ...,
         -1.52728719e-04, -8.39650660e-05,  7.03081371e-05],
        [-1.58122626e-04, -1.36002249e-04, -2.27294881e-04, ...,
         -2.64126627e-04, -2.15653877e-04,  5.15597920e-05]]),
 'b1': array([-2.34379121e-04,  1.13420713e-04, -1.66086238e-04, -3.07314809e-04,
         7.41847606e-05, -1.42979628e-05,  7.25151672e-05, -2.48615786e-04,
         5.24631725e-04, -4.99106223e-05, -1.14788694e-04, -6.102

In [44]:
from dataset.mnist import load_mnist

## MNIST 데이터 로드

In [45]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


# 하이퍼파라미터 설정

In [47]:
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

In [48]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

In [53]:
train_loss_list = []
train_acc_list = []
test_acc_list = []

In [52]:
for i in range(iters_num):
    print(i)
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 구하기
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 기울기 방향으로 learning_rate만큼 가중치 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 손실함수 값 구하기
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        # train 정확도
        train_acc = network.accuracy(x_train, t_train)
        # test 정확도
        test_acc = network.accuracy(x_test, t_test)
        
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))